In [61]:
import pandas as pd
import  numpy as np
import ast
import json
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
from sklearn.preprocessing import MultiLabelBinarizer
import ast

train = pd.read_csv('C:/Users/idan.benaun/Documents/GitHub/Kaggle_TMDB_Box_Office_Prediction/Data/train.csv')
test = pd.read_csv('C:/Users/idan.benaun/Documents/GitHub/Kaggle_TMDB_Box_Office_Prediction/Data/test.csv')
dict_columns = ['belongs_to_collection', 'genres', 'production_companies', 'cast']
mlb = MultiLabelBinarizer()

In [62]:
def convertStringToList(strVal):
    if type(strVal) is not str:
        return  []
    else:
        return ast.literal_eval(strVal)
    
def formatDictColumnAndExtractNames(strVal, col="name"):
    listOfItems = convertStringToList(strVal)
    return list(map(lambda x: x[col], listOfItems))

def extractGenres(df):
    df['genres'] = df['genres'].apply(formatDictColumnAndExtractNames)

    return df.join(pd.DataFrame(mlb.fit_transform(df.pop('genres')),
                                columns=list(map(lambda x: 'genre_'+x, mlb.classes_)),index=df.index))

def extractCommonProdCompanies(df):
    df['production_companies'] = df['production_companies'].apply(formatDictColumnAndExtractNames)

    companiesCount = df['production_companies'].apply(pd.Series).stack().value_counts()
    companiesToKeep = companiesCount[companiesCount > 30].keys()
    print("We'll keep the companies that appear more than 30 times:")
    print(companiesToKeep)

    df['production_companies'] = df['production_companies'].apply(lambda x: list(filter(lambda i: i in companiesToKeep, x)))

    return df.join(pd.DataFrame(mlb.fit_transform(df.pop('production_companies')),
                          columns=list(map(lambda x: 'prod_company_'+x,mlb.classes_)),
                          index=df.index))

def addCastLengthColumn(df):
    castNames = df['cast'].apply(formatDictColumnAndExtractNames)
    df['cast_len'] = castNames.apply(lambda x: len(x))
    df = df.drop(['cast'], axis=1, inplace=True)
    return df



df = extractGenres(train)
df = extractCommonProdCompanies(df)
df = addCastLengthColumn(df)

We'll keep the companies that appear more than 30 times:
Index(['Warner Bros.', 'Universal Pictures', 'Paramount Pictures', 'Twentieth Century Fox Film Corporation', 'Columbia Pictures', 'Metro-Goldwyn-Mayer (MGM)', 'New Line Cinema', 'Touchstone Pictures', 'Walt Disney Pictures', 'Columbia Pictures Corporation', 'TriStar Pictures', 'Relativity Media', 'Canal+', 'United Artists', 'Miramax Films', 'Village Roadshow Pictures', 'Regency Enterprises'], dtype='object')


In [63]:
df['belongs_to_collection_bool'] = np.where(df['belongs_to_collection'].isna(), 0, 1)

TypeError: 'NoneType' object is not subscriptable